# Pandas — Laboratorio 2 (CE de IA e Big Data)

**Propósito deste laboratorio:** profundizar na manipulación e análise de datos con Pandas. As **celas de código inclúen solucións**, pero **o enunciado** describe con detalle que se espera que fagas en cada paso.

**Temas:** modificación de valores, tratamento de nulos, estatística e agregacións, combinación de táboas, reestruturación (long/wide), e series temporais e datos categóricos.

Empregaremos un **dataset sintético realista** de vendas: clientes, produtos e pedidos xerados dentro do notebook para reproducibilidade.

## 0) Preparación e creación dun dataset realista
**Obxectivo:** crear tres táboas pequenas e consistentes (clientes, produtos e pedidos) que empregaremos en todo o laboratorio.

**Tarefas:**
- Fixar semente aleatoria para reproducibilidade.
- Crear `customers(customer_id, name, city, segment)` con 10 filas.
- Crear `products(product_id, category, product_name, unit_price)` con 8 filas.
- Crear `orders(order_id, date, customer_id, product_id, qty, discount)` con ~80 filas, datas entre 2024-09-01 e 2025-02-28.
- Introducir algúns nulos (en `qty` e `discount`) para practicar tratamento de valores ausentes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
pd.set_option("display.max_rows", 20)

# Táboa de clientes
customers = pd.DataFrame({
    "customer_id": range(1001, 1011),
    "name": ["Ana","Brais","Clara","Dario","Eva","Fran","Gala","Hugo","Iria","Joel"],
    "city": ["Vigo","A Coruña","Lugo","Ourense","Vigo","Santiago","Ferrol","Pontevedra","Vigo","Lugo"],
    "segment": np.random.choice(["consumer","corporate","home-office"], size=10, p=[0.6,0.25,0.15])
})

# Táboa de produtos
products = pd.DataFrame({
    "product_id": range(2001, 2009),
    "category": ["Electrónica","Electrónica","Oficina","Oficina","Hogar","Hogar","Deporte","Deporte"],
    "product_name": ["Auriculares","Teclado","Caderno","Bolígrafos","Lámpada","Tupper","Pesa 5kg","Esterilla"],
    "unit_price": [39.9, 29.5, 3.2, 1.1, 18.0, 6.5, 22.0, 15.0]
})

# Pedidos: 80 filas entre 2024-09-01 e 2025-02-28
dates = pd.date_range("2024-09-01", "2025-02-28", freq="D")
n_orders = 80
orders = pd.DataFrame({
    "order_id": range(5001, 5001+n_orders),
    "date": np.random.choice(dates, size=n_orders, replace=True),
    "customer_id": np.random.choice(customers["customer_id"], size=n_orders, replace=True),
    "product_id": np.random.choice(products["product_id"], size=n_orders, replace=True),
    "qty": np.random.randint(1, 5, size=n_orders),
    "discount": np.round(np.random.choice([0.0, 0.05, 0.10, 0.15], size=n_orders, p=[0.5,0.2,0.2,0.1]), 2)
})

# Introducimos nulos intencionados
mask = np.random.choice([True, False], size=n_orders, p=[0.1, 0.9])
orders.loc[mask, "discount"] = np.nan
mask2 = np.random.choice([True, False], size=n_orders, p=[0.05, 0.95])
orders.loc[mask2, "qty"] = np.nan

customers.head(), products.head(), orders.head()


## 1) Modificación de valores e novas columnas
**Obxectivo:** preparar unha táboa de pedidos enriquecida con información de prezo e métricas calculadas.

**Tarefas detalladas:**
1. **Enriquecer prezos:** facer `merge` entre `orders` e `products` (trátese só de traer `unit_price`).
2. **Calendario:** engadir `year` e `month` extraídos de `date`.
3. **Subtotais:** crear `subtotal = qty * unit_price` (ten en conta que `qty` pode ter nulos).
4. **Descontos:** substituír nulos en `discount` por 0 nunha nova columna `discount_filled`.
5. **Total:** calcular `total = subtotal * (1 - discount_filled)`.
6. **Verificación rápida:** amosar `head()` para comprobar resultados.

In [ ]:
# Merge para prezos e creación de novas columnas
#1


## 2) Tratamento de valores ausentes
**Obxectivo:** identificar, contar e tratar valores ausentes segundo diferentes necesidades.

**Tarefas detalladas:**
1. **Conteo de nulos por columna:** usar `isna().sum()` e visualizar só as columnas con nulos.
2. **Estratexia 1 – eliminación:** crear `orders_dropna` eliminando filas con nulos en `qty` **ou** `discount`.
3. **Estratexia 2 – substitución:** crear `orders_fillna` onde `qty` se cobre con 1 e `discount` con 0.0.
4. **Comparación:** revisar `head()` de cada versión e reflexionar sobre cando usar cada estratexia.

In [ ]:
# 2.1) Conteo de nulos



In [ ]:
# 2.2) Eliminación fronte a substitución


## 3) Operacións estatísticas e agregacións
**Obxectivo:** resumir e obter métricas útiles.

**Tarefas detalladas:**
1. `describe()` sobre `qty`, `unit_price`, `subtotal`, `total` para ter un resumo estatístico.
2. `value_counts()` de `city` (clientes) e `category` (produtos) para coñecer distribucións.
3. **Por categoría:** calcular **ingreso total** e **desconto medio**.
4. **Por mes:** usar `groupby('month').agg(...)` para obter `total_sum`, `qty_mean`, `discount_mean` e `orders_count`.
5. **Describe por grupo:** combinar `groupby('category')` con `describe()` sobre `total`.
6. **Correlación:** matriz de correlación entre `qty`, `unit_price`, `subtotal` e `total`.

In [ ]:
# 3.1) describe()



In [ ]:
# 3.2) value_counts por city e category



In [ ]:
# 3.3) ingreso total e desconto medio por categoría



In [ ]:
# 3.4) agregación por mes


In [ ]:
# 3.5) describe por grupo


In [ ]:
# 3.6) correlación



## 4) Combinación de datos de varias táboas (merge, concat)
**Obxectivo:** unir información entre táboas e combinar rexistros.

**Tarefas detalladas:**
1. **Merge con customers:** enriquecer `orders_enriched` con `city` e `segment`. Logo calcular o **ingreso medio por cidade**.
2. **Concat de novos pedidos:** crear unha pequena mostra `orders_new` (5 filas) e concatenala con `orders_enriched`; asegurarse de eliminar duplicados por `order_id`.

In [ ]:
# 4.1) MERGE con customers e ingreso medio por cidade



In [ ]:
# 4.2) CONCAT con pedidos novos e eliminación de duplicados



## 5) Reestruturación (long/wide, pivot, stack/unstack)
**Obxectivo:** cambiar a forma dos datos para análises alternativas.

**Tarefas detalladas:**
1. **Formato long con `melt`:** transformar `qty` e `total` en filas, mantendo `month` e `category` como id.
2. **Pivot table:** obter unha táboa co **índice** `month`, **columnas** `category` e **valores** a suma de `total`.
3. **Stack / Unstack:** practicar a ida e volta a partir do pivot anterior.

In [ ]:
# 5.1) melt (wide -> long)



In [ ]:
# 5.2) pivot table (long -> wide)



In [ ]:
# 5.3) stack/unstack



## 6) Series temporais e datos categóricos
**Obxectivo:** traballar con datas e optimizar variables categóricas.

**Tarefas detalladas:**
1. **Mensualización:** converter `date` en índice temporal ordenado e facer `resample('M').sum()` de `total`.
2. **Categóricos:** converter `category` e `segment` a tipo `category` e comparar consumo de memoria **antes** e **despois**.
3. **Gráfica mensual:** debuxar a serie mensual de ingresos nun **único gráfico** con títulos e eixes.

In [ ]:
# 6.1) Resample mensual


In [ ]:
# 6.2) Conversión a categóricos e memoria


In [ ]:
# 6.3) Visualización mensual
